# Modelos Lineales Generalizados en Python
# Regresión multinomial - ordinal

<img src="https://raw.githubusercontent.com/fhernanb/fhernanb.github.io/master/docs/logo_unal_color.png" alt="drawing" width="200"/>

Aquí se muestran varios ejemplos de como usar Python para ajustar un modelo lineal generalizado. 

Las librerías necesarias son las siguientes:

In [1]:
from sklearn.linear_model import LogisticRegression

from sklearn.preprocessing import OneHotEncoder

Otras librerías que se usarán en los ejemplos son:

In [2]:
import pandas as pd

## Ejemplo

En esta actividad vamos a utilizar los datos presentados la sección 6.2.2 del libro "An Introduction to Categorical Data Analysis" de Agresti (2019). El objetivo es ajustar el siguiente modelo:

$$
logit(P(Y \leq j)) = \alpha_j + \beta_1 Partido_{repub} + \beta_2 Genero_{male}, \, \text{para} \, j=1, 2, 3, 4
$$

El objetivo es modelar ajustar un modelo de regresión multinomial para explicar la variable respuesta ideología política $Y$ en función del género y el partido. 

Abajo una figura ilustrativa.

<img src="logo_partidos_politicos.jpg" alt="drawing" width="800"/>

Lo primero que usted debe hacer es leer la base de datos.

In [3]:
datos = pd.read_csv('political_ideology.txt', sep='\s+', header=0)
datos.head()

,genero,partido,ideologia
1,Female,Democ,Muy lib
2,Female,Democ,Muy lib
3,Female,Democ,Muy lib
4,Female,Democ,Muy lib
5,Female,Democ,Muy lib


Para ver el tamaño de la base de datos

In [4]:
datos.shape

(661, 3)

Vamos a explorar el tipo de cada una de las variables.

In [5]:
datos.dtypes

genero       object
partido      object
ideologia    object
dtype: object

Vamos a crear la matriz $X$ para entrenar el modelo.

In [6]:
X = datos[["genero", "partido"]]

enc = OneHotEncoder(drop="first", sparse=False)
enc = enc.fit(X)
X = enc.transform(X)

print("La matriz X es:")
print(X)
print("\n")
print("Los datos originales son:")
print(datos[["genero", "partido"]])

La matriz X es:
[[0. 0.]
 [0. 0.]
 [0. 0.]
 ...
 [1. 1.]
 [1. 1.]
 [1. 1.]]


Los datos originales son:
     genero partido
1    Female   Democ
2    Female   Democ
3    Female   Democ
4    Female   Democ
5    Female   Democ
..      ...     ...
657    Male   Repub
658    Male   Repub
659    Male   Repub
660    Male   Repub
661    Male   Repub

[661 rows x 2 columns]


Vamos a codificar la variable ideología para crear el vector $y$.

In [7]:
scale_mapper = {"Muy lib":1, "Algo lib":2, "Mod":3, "Algo conser":4, "Muy conser":5}
datos["y"] = datos["ideologia"].replace(scale_mapper)

y = datos["y"]

datos

,genero,partido,ideologia,y
1,Female,Democ,Muy lib,1
2,Female,Democ,Muy lib,1
3,Female,Democ,Muy lib,1
4,Female,Democ,Muy lib,1
5,Female,Democ,Muy lib,1
...,...,...,...,...
657,Male,Repub,Muy conser,5
658,Male,Repub,Muy conser,5
659,Male,Repub,Muy conser,5
660,Male,Repub,Muy conser,5


## Ajustando el modelo

In [8]:
# define the multinomial logistic regression model
model = LogisticRegression(multi_class='multinomial', solver='lbfgs')

# fit the model on the whole dataset
model.fit(X, y)

LogisticRegression(multi_class='multinomial')

## Haciendo predicciones con el modelo ajustado

¿Cuál podría ser la ideología de una mujer republicana?

In [9]:
# define a single row of input data
row = [0, 1]

# predict a multinomial probability distribution
probhat = model.predict_proba([row])

# predict the class
yhat = model.predict([row])

# summarize the results
print('Predicted Probabilities: %s' % probhat[0])
print('Predicted Class: %s' % yhat[0])

Predicted Probabilities: [0.00780786 0.03266093 0.12941612 0.602974   0.22714109]
Predicted Class: 4


¿Cuál podría ser la ideología de un hombre demócrata?

In [10]:
row = [1, 0]

# predict a multinomial probability distribution
probhat = model.predict_proba([row])

# predict the class
yhat = model.predict([row])

# summarize the results
print('Predicted Probabilities: %s' % probhat[0])
print('Predicted Class: %s' % yhat[0])

Predicted Probabilities: [0.11837012 0.44207927 0.28409692 0.12852072 0.02693296]
Predicted Class: 2


In [11]:
# Para conocer los coeficientes del modelo ???
print(model.intercept_)
print('\t')
print(model.coef_)

[-0.2886596   1.18936971  0.95245433 -0.08206901 -1.77109543]
	
[[ 0.12966351 -2.10256699]
 [-0.03069291 -2.14954801]
 [-0.23595129 -0.5357792 ]
 [ 0.00534673  2.03758525]
 [ 0.13163396  2.75030895]]


In [12]:
# Para crear la matriz de confusión
mat_conf = pd.crosstab(index=model.predict(X), columns=y, rownames=['Prediccion'], colnames=['Membresia'])
print(mat_conf)

Membresia    1    2    3    4   5
Prediccion                       
2           45  178  129   48   7
4            0    6   29  155  64


In [13]:
# Para obtener el accuracy manualmente
(178 + 155) / 661

0.5037821482602118

In [14]:
# Return the mean accuracy on the given test data and labels.
model.score(X, y)

0.5037821482602118